**Using VGG16**

In [ ]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/MyDrive/archive (1).zip"

Archive:  /content/drive/MyDrive/archive (1).zip
  inflating: test_data/test_data/blasti/DSC_6396.jpg  
  inflating: test_data/test_data/blasti/DSC_6397.jpg  
  inflating: test_data/test_data/blasti/DSC_6398.jpg  
  inflating: test_data/test_data/blasti/DSC_6399.jpg  
  inflating: test_data/test_data/blasti/DSC_6400.jpg  
  inflating: test_data/test_data/blasti/DSC_6401.jpg  
  inflating: test_data/test_data/blasti/DSC_6402.jpg  
  inflating: test_data/test_data/blasti/DSC_6403.jpg  
  inflating: test_data/test_data/blasti/DSC_6405.jpg  
  inflating: test_data/test_data/blasti/DSC_6406.jpg  
  inflating: test_data/test_data/blasti/DSC_6407.jpg  
  inflating: test_data/test_data/blasti/DSC_6408.jpg  
  inflating: test_data/test_data/blasti/DSC_6409.jpg  
  inflating: test_data/test_data/blasti/DSC_6410.jpg  
  inflating: test_data/test_data/blasti/DSC_6411.jpg  
  inflating: test_data/test_data/bonegl/DSC_4587.jpg  
  inflating: test_data/test_data/bonegl/DSC_4588.jpg  
  inflating: tes

In [ ]:
train_path = "/content/train_data"
test_path = "/content/test_data"

In [ ]:
train_gen = ImageDataGenerator(rescale=1./255, shear_range= 0.2, zoom_range=0.2,horizontal_flip=True)
test_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train = train_gen.flow_from_directory(train_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

test = test_gen.flow_from_directory(test_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

Found 150 images belonging to 1 classes.
Found 157 images belonging to 1 classes.


In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [ ]:
#adding the preprocessing layer to the front of vgg

vgg = VGG16(include_top=False, weights='imagenet',input_shape=(224,244,3))

58889256/58889256 [==============================] - 3s 0us/step


In [ ]:
#train model with existing weights

for layer in vgg.layers:
  print(layer)

In [ ]:
#training the model with existing weights
for layer in vgg.layers:
  layer.trainable=False

In [ ]:
x= Flatten()(vgg.output)

In [ ]:
#output layer
prediction = Dense(3,activation='softmax')(x)

In [ ]:
#create vgg16 model
model = Model(inputs=vgg.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 244, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 244, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 244, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 122, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 122, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 122, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 61, 128)       0   

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit_generator(train, validation_data=test,epochs=5, steps_per_epoch=len(train),validation_steps=len(test))

**Using ResNet50**

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
resnet = ResNet50(include_top=False,input_shape=(224,224,3))

94765736/94765736 [==============================] - 4s 0us/step


In [ ]:
for layer in resnet.layers:
  layer.trainable = False


In [ ]:
for layer in resnet.layers:
  print(layer)

In [ ]:
y = Flatten()(resnet.output)

In [ ]:
out = Dense(3, activation='softmax')(y)

In [ ]:
res_model = Model(inputs=resnet.input,outputs=out)

In [ ]:
res_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [ ]:
res_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
res_model.fit(train,epochs=5,validation_data=test,steps_per_epoch=len(train),
              validation_steps=len(test))

Epoch 1/5
7/7 [==============================] - 114s 18s/step - loss: 4.8994 - accuracy: 1.0000 - val_loss: 4.9784 - val_accuracy: 1.0000
Epoch 2/5
7/7 [==============================] - 133s 21s/step - loss: 4.9200 - accuracy: 1.0000 - val_loss: 4.9056 - val_accuracy: 1.0000
Epoch 3/5
7/7 [==============================] - 133s 21s/step - loss: 4.9075 - accuracy: 1.0000 - val_loss: 4.8537 - val_accuracy: 1.0000
Epoch 4/5
7/7 [==============================] - 133s 21s/step - loss: 4.8835 - accuracy: 1.0000 - val_loss: 4.8176 - val_accuracy: 1.0000
Epoch 5/5
7/7 [==============================] - 133s 20s/step - loss: 4.8929 - accuracy: 1.0000 - val_loss: 4.9258 - val_accuracy: 1.0000
